# Trabalho Prático 1 - Grupo 15
#### João Gonçalves - pg46535
#### Sara Queirós - pg47661
## Exercício 2 

2. Use o SageMath para, 
    1. Construir uma classe Python que implemente um KEM- RSA. A classe deve
        1. Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits do módulo RSA) e gere as chaves pública e privada.
        2. Conter funções para encapsulamento e revelação da chave gerada.
    2. Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.


In [1]:
import os
import random
import hashlib
from sage.arith.power import generic_power

In [3]:
class KEMRSA():
    def __init__(self, param, random):
        #parametro de segurança
        self.p = 0
        self.q = 0
        self.n = 0
        self.e = 0
        self.d = 0
        self.to_key = None
        self.size = param
        self.chave_cifrada = None
        self.chave_decifrada = None
        self.chave_gerada_e = None
        self.chave_gerada_d = None
        self.salt = random
        self.private_key = None
        self.public_key  = None

    def generate_keys(self):

        #temos de encontrar p e q primos com size/2 bits
        while not (self.p in Primes()):
            self.p = next_prime(ZZ.random_element(pow(2,self.size/2-1)+1, pow(2,self.size/2)-1))

        while not (self.q in Primes()):
            self.q = next_prime(ZZ.random_element(pow(2,self.size/2-1)+1, pow(2,self.size/2)-1))

        #calcular o n 
        self.n = self.p * self.q
        
        #calcular o phi
        phi = (self.p -1) * (self.q-1)

        #e is a pseudo-random integer
        self.e = ZZ.random_element(phi)

        while (gcd(e,phi)!=1):
            self.e = ZZ.random_element(phi)

        #we use the extended Euclidean algorithm to calculate d 
        bezout = xgcd(self.e,phi)
        self.d = Integer(mod(bezout[1], phi))

        if not (mod(self.d*self.e, phi) == 1):
            self.generate_keys()        
              
        #Assim temos todos os parâmetros para as chaves privadas e públicas 
        self.private_key = (self.n, self.d)
        self.public_key  = (self.n, self.e)


    #Encapsular a chave 
    def encapsulamento(self, chave, second_public_key):
        self.to_key = int(chave)
        #Gera o salt
        key_to_bytes = self.to_key.to_bytes(int(self.size/8), "big")
        #Aplicar a função de hash de sha256
        self.chave_gerada_e = hashlib.pbkdf2_hmac('sha256', key_to_bytes, self.salt, 100000)
        #cifrar a chave publica do outro interveniente
        self.cifrar(second_public_key)
        return self.chave_cifrada, self.chave_gerada_e

    def cifrar(self, second_public_key):
        n , e = second_public_key
        self.chave_cifrada = power_mod(self.to_key, e, n)

    def decifrar(self, msg_to_decifrar):
        self.chave_cifrada = msg_to_decifrar
        key_to_dec = int(self.chave_cifrada)
        self.chave_decifrada = power_mod(key_to_dec, self.d, self.n)
    
    #Decifrar a mensagem recebida 
    def revelar(self, msg_to_decifrar):
        self.decifrar(msg_to_decifrar)
        #Converter a chave recebida para bytes
        key_to_bytes = int(self.chave_decifrada).to_bytes(int(self.size/8), "big")
        #aplicar a função de hash para decifrar 
        self.chave_gerada_d = hashlib.pbkdf2_hmac('sha256', key_to_bytes, self.salt, 100000)
        return self.chave_decifrada, self.chave_gerada_d

    #Verificar se a chave do receiver e do sender coincidem 
    def verificar(self, receiver, sender):
        self.chave_gerada_e = receiver
        self.chave_gerada_d = sender
        if (self.chave_gerada_e == self.chave_gerada_d):
            print("As chaves coincidem! Mensagem Inicial intacta")
        else: 
            print("As chaves não coincidem!")

# Exercício a)
A classe KEMRSA inicializa-se com 1024 bits.

A partir daí gera uma chave privada e respetivamente a pública.

Dado um elemento aleatório, o "receiver" encapsula com esse elemento a chave pública do "sender".

Após encapsular, devolve a chave que deve ser comum e a mensagem cifrada.

O outro interveniente recebe e decifra a mensagem.

No fim, verifica-se se as chaves são iguais, caso a mensagem se verifique intacta.

As funções de encapsulamento utilizam o SHA256 para cifrar a chave que é enviada.


In [4]:
random = os.urandom(16)
kem_first = KEMRSA(1024, random)
kem_second = KEMRSA(1024, random)

kem_first.generate_keys()
kem_second.generate_keys()

elem = ZZ.random_element(1024)

msg_enc, sender_key = kem_first.encapsulamento(elem, kem_second.public_key)
dec, receiver_key = kem_second.revelar(msg_enc)

kem_first.verificar(sender_key, receiver_key)

print("Chave de a: ", sender_key)
print("Chave de b: ", receiver_key)


As chaves coincidem! Mensagem Inicial intacta
Chave de a:  b'Jy6\x06T\xfc4u\xadm\x18w\xbel\x948eG\x14\xc4\xc4I\xd5\xb3[\xad\xb0\xb1\x80x\xfd\xb2'
Chave de b:  b'Jy6\x06T\xfc4u\xadm\x18w\xbel\x948eG\x14\xc4\xc4I\xd5\xb3[\xad\xb0\xb1\x80x\xfd\xb2'
